### Examples of SPARQL queries using the SPARQL Jupyter kernel
##### Dependencies
[OpenLink Virtuoso SPARQL Processor](https://virtuoso.openlinksw.com)<br>
[SPARQL Jupyter Kernel](https://github.com/paulovn/sparql-kernel)

### Example 1
The *%endpoint* magic provides for setting of a SPARQL endpoint.

In [ ]:
%endpoint http://scc-dean1.lancs.ac.uk:8890/sparql

### Example 2
Discover all monitoring stations in "Lune and Wyre" catchment area along and display the *rt:town, rdfs:label, geo:long, geo:lat, rt:measures and rt:latestReading* property values.

In [ ]:
PREFIX rt:<http://environment.data.gov.uk/flood-monitoring/def/core/>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX geo:<http://www.w3.org/2003/01/geo/wgs84_pos#>

SELECT ?label ?town ?lon ?lat ?measures ?latestValue
FROM <http://environment.data.gov.uk/flood-monitoring/id/stations.rdf>
FROM <http://environment.data.gov.uk/flood-monitoring/id/measures.rdf>
FROM <http://environment.data.gov.uk/flood-monitoring/data/readings.rdf>
WHERE 
{
  ?s rt:town ?town ;
     geo:long ?lon ;
     geo:lat ?lat ;
     rt:catchmentName "Lune and Wyre" ; 
     rdfs:label ?label ;
     rt:measures ?measures . 
     ?measures rt:latestReading ?latestReading . 
     ?latestReading rt:value ?latestValue .
}

### Example 3
Find the "Morecambe North" and "Morecambe South" Bathing Water Quality (BWQ) stations using the Environment Agency BWQ endpoint and the *rdfs:label* property.

In [ ]:
%endpoint http://environment.data.gov.uk/sparql/bwq/query

In [ ]:
PREFIX bw: <http://environment.data.gov.uk/def/bathing-water/>
PREFIX bwq: <http://environment.data.gov.uk/def/bathing-water-quality/>
PREFIX ef-sp: <http://location.data.gov.uk/def/ef/SamplingPoint/>
PREFIX interval: <http://reference.data.gov.uk/def/intervals/>
PREFIX stats: <http://statistics.data.gov.uk/def/administrative-geography/>
PREFIX sr: <http://data.ordnancesurvey.co.uk/ontology/spatialrelations/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?s
WHERE 
{
  { ?s rdfs:label "Morecambe North"@en . }
  UNION
  { ?s rdfs:label "Morecambe South"@en . }
}


### Example 4
Discover any BWQ stations with Morecambe in their name using the *rdfs:label* property and a regular expression filter.

In [ ]:
%endpoint http://environment.data.gov.uk/sparql/bwq/query

In [ ]:
prefix bw: <http://environment.data.gov.uk/def/bathing-water/>
prefix bwq: <http://environment.data.gov.uk/def/bathing-water-quality/>
prefix ef-sp: <http://location.data.gov.uk/def/ef/SamplingPoint/>
prefix interval: <http://reference.data.gov.uk/def/intervals/>
prefix stats: <http://statistics.data.gov.uk/def/administrative-geography/>
prefix sr: <http://data.ordnancesurvey.co.uk/ontology/spatialrelations/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>

select ?label ?lat ?lon
where
{ ?o ef-sp:samplingPoint ?sp .
  ?sp geo:lat ?lat .
  ?sp geo:long ?lon .
  { select ?o ?label
    where {
      ?o a bw:BathingWater ;
         rdfs:label ?label .
      filter regex(?label, "morecambe", "i")
    }
  }
}

### Example 5
Discover DBpedia information about any resources within 20km of New York City and display the *rdfs:label* " English language property value.

In [ ]:
%endpoint http://dbpedia.org/sparql

In [ ]:
SELECT DISTINCT ?resource 
WHERE 
  { 
    <http://dbpedia.org/resource/New_York_City> geo:geometry ?sourcegeo . 
    ?resource geo:geometry ?location ; 
                                 rdfs:label ?label . 
    FILTER( bif:st_intersects( ?location, ?sourcegeo, 20 ) ) . 
    FILTER( lang( ?label ) = "en" ) 
  }

### Example 6
Discover DBpedia information about any resources within 5km of Lancaster using a federated query using the *http://dbpedia.org/sparql* service.

In [ ]:
%endpoint http://scc-dean1.lancs.ac.uk:8890/sparql

In [ ]:
SELECT DISTINCT ?resource ?location
WHERE 
{ 
    SERVICE <http://dbpedia.org/sparql>
    {
        ?resource geo:geometry ?location ;
                  rdfs:label ?label . 
        FILTER( bif:st_intersects( ?location, bif:st_point(-2.78555560112, 54.010276794434), 5)) .
    }
}

### Example 7
Explore Virtuoso geospatial reasoning capabilities and display output in N3 format. The example finds upto 10 locations within a boundary defined by a polygon.

In [ ]:
%format N3

In [ ]:
SELECT DISTINCT ?resource ?label ?long ?lat
WHERE 
{ 
    SERVICE <http://dbpedia.org/sparql>
    {
        ?resource geo:geometry ?location ;
                  rdfs:label ?label . 
        FILTER( bif:st_within( ?location, bif:st_geomfromtext("POLYGON((-2.980620861053467 54.10704331093268, -2.685363292694092 54.07563161821085, -2.708709239959717 53.979638627170516, -2.9559016227722164 54.00063080536647, -2.980620861053467 54.10704331093268 ))"))) .
        BIND(bif:st_x(?location) AS ?long) .
        BIND(bif:st_y(?location) AS ?lat) .
    }
}
LIMIT 10

### Example 8
Find monitoring stations within 10km of a geospatial point using the Environment Agency Flood Monitoring APIs.

In [ ]:
PREFIX rt:<http://environment.data.gov.uk/flood-monitoring/def/core/>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX geo:<http://www.w3.org/2003/01/geo/wgs84_pos#>

SELECT ?label ?town ?lon ?lat ?s 
FROM <http://environment.data.gov.uk/flood-monitoring/id/stations.rdf>
FROM <http://environment.data.gov.uk/flood-monitoring/id/measures.rdf>
FROM <http://environment.data.gov.uk/flood-monitoring/data/readings.rdf>
WHERE {
  ?s rt:town ?town ;
     geo:long ?lon ;
     geo:lat ?lat ; 
     rdfs:label ?label .
     FILTER( bif:st_intersects(bif:st_point(?lon, ?lat), bif:st_point(-2.78555560112, 54.010276794434), 10.0)) .
}